<a href="https://colab.research.google.com/github/rjnakano/manejo-inventarios-pdg/blob/main/Notebooks/clusterboot050621V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade kmodes

Requirement already up-to-date: kmodes in /usr/local/lib/python3.7/dist-packages (0.11.0)


In [2]:
from google.colab import drive
import pathlib
import pandas as pd
import numpy as np
from pylab import *

from sklearn.preprocessing import StandardScaler

from kmodes.kprototypes import KPrototypes

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx

import seaborn as sns
%matplotlib inline
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()

from plotnine import *
import plotnine

In [3]:
plot_kwds = {'alpha' : 0.5, 's' : 1, 'linewidths':0}

In [4]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
# Dataset de inventarios sin dimensiones de los productos
data1 = pd.read_csv("/content/gdrive/MyDrive/TDGdata/dfstd050621.csv")

In [6]:
data1['AISLE']=data1['AISLE'].astype(str)

In [7]:
data1.dtypes

AISLE        object
IG           object
HTS          object
PLANNING     object
ABC          object
UOM          object
CAT          object
COSTX       float64
QTYVAR      float64
dtype: object

In [8]:
data1.head(5)

,AISLE,IG,HTS,PLANNING,ABC,UOM,CAT,COSTX,QTYVAR
0,17,IG56,HTS119,1/1,B,PC,CAT17,-0.029976,-0.053250
1,17,IG56,HTS119,1/1,B,PC,CAT17,0.092472,-0.022999
2,17,IG56,HTS119,1/1,C,PC,CAT17,0.266130,0.004373
3,17,IG56,HTS119,1/1,B,PC,CAT17,-0.025112,-0.060323
4,15,IG56,HTS119,1/1,C,PC,CAT17,-0.077982,-0.062534


In [9]:
# scikit-learn bootstrap
from sklearn.utils import resample

In [10]:
data1.shape[0]

16239

In [11]:
catColumnsPos = [data1.columns.get_loc(col) for col in list(data1.select_dtypes('object').columns)]

In [12]:
catColumnsPos

[0, 1, 2, 3, 4, 5, 6]

In [13]:
# Fit the cluster 
k = 2
gamma = 0.5
import time
start_time = time.time()
kprototype = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 0)
kprototype.fit_predict(data1, categorical = catColumnsPos)
print("--- %s seconds ---" % (time.time() - start_time))

--- 68.24498271942139 seconds ---


In [14]:
data1['Cluster'] = kprototype.labels_

In [15]:
def add2set (nuevos):
  conjunto = {999999}
  conjunto.update(nuevos)
  conjunto.discard(999999)
  return(conjunto)


In [16]:
orig0 = {}
orig1 = {}

In [17]:
orig0 = add2set(data1[data1['Cluster']==0].index)
orig1 = add2set(data1[data1['Cluster']==1].index)

In [18]:
data1.drop('Cluster', inplace=True, axis=1)

In [19]:
# Inicializamos la matriz de distancias con ceros
import time
start_time = time.time()
nBootstrap = 100
gamma = 0.5
jaccard = [ [ 0 for i in range(k) ] for j in range(nBootstrap) ]
boot0 = {}
boot1 = {}

In [20]:
pd.options.mode.chained_assignment = None

In [21]:
for i in range(nBootstrap):
  print('bootstrap # ',i);
  boot = resample(data1, replace=True, n_samples=data1.shape[0], random_state=i)
  # Fit the cluster
  kprototypebs = KPrototypes(n_jobs = -1, n_clusters = k, init = 'Huang', gamma = gamma, random_state = 1)
  kprototypebs.fit_predict(boot, categorical = catColumnsPos)
  boot['Cluster'] = kprototypebs.labels_
  print("--- %s seconds ---" % (time.time() - start_time));
  boot0 = add2set(boot[boot['Cluster']==0].index)
  boot1 = add2set(boot[boot['Cluster']==1].index)
  jaccard[i][0]=max(len(orig0 & boot0) / len(orig0 | boot0), len(orig0 & boot1) / len(orig0 | boot1))
  jaccard[i][1]=max(len(orig1 & boot0) / len(orig1 | boot0), len(orig1 & boot1) / len(orig1 | boot1))
  boot.drop('Cluster', inplace=True, axis=1)
  print(jaccard[i][0:k]);

bootstrap #  0
--- 54.62896680831909 seconds ---
[0.6092843326885881, 0.6328904710444345]
bootstrap #  1
--- 105.52250838279724 seconds ---
[0.586073500967118, 0.6288600838734274]
bootstrap #  2
--- 159.4992344379425 seconds ---
[0.6150870406189555, 0.6322138276934798]
bootstrap #  3
--- 204.87433862686157 seconds ---
[0.5783365570599613, 0.6347229283172343]
bootstrap #  4
--- 256.4835968017578 seconds ---
[0.6312741312741312, 0.6301361149980919]
bootstrap #  5
--- 306.64420437812805 seconds ---
[0.6354961832061069, 0.6320508744038156]
bootstrap #  6
--- 357.19520831108093 seconds ---
[0.5996131528046421, 0.630809333079026]
bootstrap #  7
--- 407.45271492004395 seconds ---
[0.5846702317290553, 0.6335707925200356]
bootstrap #  8
--- 459.06002163887024 seconds ---
[0.6365384615384615, 0.6336980027986261]
bootstrap #  9
--- 508.35796189308167 seconds ---
[0.6305609284332688, 0.6388959552276774]
bootstrap #  10
--- 556.6794764995575 seconds ---
[0.031243839936920953, 0.6003491489494357]
bo

In [22]:
pd.options.mode.chained_assignment = 'warn'

In [27]:
jaccard = pd.DataFrame(jaccard)

In [28]:
jaccard.describe()

,0,1
count,100.000000,100.000000
mean,0.555371,0.612528
std,0.172055,0.069502
min,0.030806,0.313573
25%,0.583404,0.628307
50%,0.613681,0.630163
75%,0.634429,0.632405
max,0.680851,0.638896


In [29]:
jaccard.to_csv("/content/gdrive/MyDrive/TDGdata/jaccard050721.csv", index = False)